# 0. Configs and Imports

## 0.1. Imports

In [7]:
# Import for directory creation
from src.utils.file_operators import create_directory_structure

# Imports for data fetch
from src.data_collectors.yahoo_fin_collector import fetch_data

# Imports for loading config
from src.utils.file_operators import load_yaml

# Imports for Correlation Calculation
from src.computation.correlation import calculate_pairwise_corrrelation

# Imports for Portfolio Unierse Generation
from src.computation.ticker_universe_generator import create_portfolio_universe

## 0.2. Base file path, Configuration Filepath & Configuration Dictionary

In [8]:
base_file_path = "C:/Users/sharv/Documents/Sharvil/Projects/AutoWeight/data"

In [9]:
conf_path = "C:/Users/sharv/Documents/Sharvil/Projects/AutoWeight/config-sd.yaml"

In [10]:
config = load_yaml(conf_path)

## 0.3. Create Directory Structure (Optional: Run once only)

In [11]:
create_directory_structure(path=base_file_path, verbose=True)

INFO: Creating Base Directory
INFO: Base Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data
INFO: Creating Sub Directories
INFO: Sub Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data\01_raw

INFO: Sub Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data\02_processed

INFO: Sub Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data\03_analysis

SUCCESS: Created all directories successfully!



# 1. Data Fetch

## 1.1. Validate the configurations for data fecth

In [12]:
# Print the tickers defined in the config
ticker_list = config['config']['data']['tickers']

# Print the window and volitility_indicator defined in the config
window = config['config']['data']['window_in_days']
volatility_indicator = config['config']['data']['vix']

In [13]:
print("The tickers in the config list are:")

for idx, ticker in enumerate(ticker_list):
    print(f"    {idx+1}. {ticker}")

print(f"The window of data to evaluate is `{window}` days")

print(f"The volatility indicator used is `{volatility_indicator}`")

The tickers in the config list are:
    1. NIFTYBEES.NS
    2. NETFCONSUM.NS
    3. BANKBEES.NS
    4. GOLDBEES.NS
    5. HNGSNGBEES.NS
    6. MID150BEES.NS
    7. MAN50ETF.NS
    8. MANXT50.NS
    9. AXISGOLD.NS
    10. AXISNIFTY.NS
    11. EBBETF0423.NS
    12. EBBETF0425.NS
    13. EBBETF0430.NS
    14. EBBETF0431.NS
    15. ABSLBANETF.NS
    16. BSLGOLDETF.NS
    17. BSLNIFTY.NS
    18. ABSLNN50ET.NS
    19. CPSEETF.NS
    20. LIQUIDETF.NS
    21. EQ30.NS
    22. ICICINF100.NS
    23. ICICINIFTY.NS
    24. ICICINV20.NS
    25. ICICISENSX.NS
    26. ICICIBANKP.NS
    27. ICICI500.NS
    28. IDBIGOLD.NS
    29. IDFNIFTYET.NS
    30. IBMFNIFTY.NS
    31. IVZINGOLD.NS
    32. IVZINNIFTY.NS
    33. KOTAKBKETF.NS
    34. KOTAKGOLD.NS
    35. KOTAKNIFTY.NS
    36. KOTAKNV20.NS
    37. KOTAKPSUBK.NS
    38. LICNFNHGP.NS
    39. LICNETFN50.NS
    40. LICNETFSEN.NS
    41. LICNETFGSC.NS
    42. M50.NS
    43. M100.NS
    44. N100.NS
The window of data to evaluate is `900` days
The volatility

## 1.2. Run the `fetch_data` pipeline

In [14]:
ohlcv_data = fetch_data(
    config_path=conf_path,
    save_data=True, 
    verbose=False
)

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: NETFCONSUM.NS"}}}
$NETFCONSUM.NS: possibly delisted; no timezone found

1 Failed download:
['NETFCONSUM.NS']: possibly delisted; no timezone found


ERROR: Possible Delising or Incorrect Name for ticker: NETFCONSUM.NS



$MAN50ETF.NS: possibly delisted; no timezone found

1 Failed download:
['MAN50ETF.NS']: possibly delisted; no timezone found


ERROR: Possible Delising or Incorrect Name for ticker: MAN50ETF.NS



$MANXT50.NS: possibly delisted; no timezone found

1 Failed download:
['MANXT50.NS']: possibly delisted; no timezone found


...

ERROR: Possible Delising or Incorrect Name for ticker: M50.NS



$M100.NS: possibly delisted; no timezone found

1 Failed download:
['M100.NS']: possibly delisted; no timezone found


ERROR: Possible Delising or Incorrect Name for ticker: M100.NS



$N100.NS: possibly delisted; no timezone found

1 Failed download:
['N100.NS']: possibly delisted; no timezone found


ERROR: Possible Delising or Incorrect Name for ticker: N100.NS

SUCCESS: fetch_data pipeline run completed


In [15]:
ohlcv_data['close']

Ticker,NIFTYBEES.NS,BANKBEES.NS,GOLDBEES.NS,HNGSNGBEES.NS,MID150BEES.NS,AXISGOLD.NS,AXISNIFTY.NS,EBBETF0425.NS,EBBETF0430.NS,EBBETF0431.NS,...,CPSEETF.NS,LIQUIDETF.NS,IDFNIFTYET.NS,IBMFNIFTY.NS,IVZINGOLD.NS,IVZINNIFTY.NS,LICNFNHGP.NS,LICNETFN50.NS,LICNETFSEN.NS,LICNETFGSC.NS
Date,,,,,,,,,,,,,,,,,,,,,
2023-09-13,221.429993,467.510010,49.880001,268.350006,154.479996,50.160000,214.979996,1152.089966,1298.869995,1158.430054,...,51.580002,999.98999,214.539993,200.649994,5223.950195,2227.550049,214.089996,217.500000,721.650024,23.940001
2023-09-14,221.529999,468.570007,49.880001,269.329987,155.740005,50.049999,215.399994,1152.839966,1301.709961,1159.219971,...,52.139999,999.98999,216.309998,200.770004,5195.399902,2237.979980,214.759995,218.240005,738.739990,23.950001
2023-09-15,222.570007,470.739990,50.119999,271.450012,155.539993,50.259998,216.029999,1152.890015,1301.150024,1158.430054,...,51.910000,999.98999,216.000000,201.600006,5215.000000,2237.979980,214.990005,218.679993,742.049988,24.040001
2023-09-18,221.919998,468.589996,50.439999,269.899994,155.119995,50.639999,215.770004,1154.239990,1298.680054,1156.900024,...,52.709999,999.98999,215.899994,201.000000,5240.100098,2237.979980,214.350006,219.169998,739.809998,24.010000
2023-09-20,219.600006,462.679993,50.380001,268.540009,154.550003,50.650002,213.710007,1153.630005,1299.770020,1157.800049,...,53.119999,999.98999,213.820007,200.220001,5267.950195,2215.649902,212.990005,216.820007,733.599976,24.030001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-23,290.450012,631.599976,130.000000,542.380005,225.789993,131.210007,282.429993,NaN,1572.010010,1406.020020,...,101.940002,1000.00000,281.600006,NaN,13687.849609,2929.360107,286.359985,285.739990,936.530029,28.840000
2026-02-24,287.850006,629.960022,131.220001,538.580017,225.520004,132.479996,279.950012,NaN,1573.109985,1400.449951,...,102.669998,1000.00000,279.720001,NaN,13796.450195,2904.330078,285.000000,283.140015,936.530029,28.830000
2026-02-25,288.630005,629.489990,131.690002,536.710022,226.220001,132.830002,280.540009,NaN,1573.329956,1401.130005,...,103.320000,1000.00000,279.820007,NaN,13843.599609,2895.500000,286.429993,284.790009,925.809998,28.910000


# 2. Calculate Correlation

## 2.1. Validate the correaltion parameters

In [16]:
correlation_params = config['config']['parameters']['correlation']

In [17]:
print(f"The method used for calculating correlation is: {correlation_params['method']}")
print(f"Minimum samples needed for correlation are: {correlation_params['min_observations']}")
print(f"The price metrics defined inn the configs are:")
for idx, metric in enumerate(correlation_params['metrics']):
    print(f"    {idx+1}. {metric}")


The method used for calculating correlation is: spearman
Minimum samples needed for correlation are: 60
The price metrics defined inn the configs are:
    1. open
    2. high
    3. low
    4. close
    5. adj_close


## 2.2. Run the `calculate_pairwise_corrrelation` pipeline

In [18]:
correlation_results = calculate_pairwise_corrrelation(
    config_path=conf_path,
    save_data=True,
    verbose=True
)

INFO:    Starting Correlation Pipeline for open metric of data
INFO:    Calculating PCT Change for open metric of data
INFO:    Calculating Correlation for PCT Change for open metric of data


C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\src\computation\correlation.py:26: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return df.pct_change()


INFO:    Adjusting Correlation based on optimization strategy for open metric of data
SUCCESS:    Correlation Pipeline for open metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for open metric of data finished successfully

INFO:    Starting Correlation Pipeline for high metric of data
INFO:    Calculating PCT Change for high metric of data
INFO:    Calculating Correlation for PCT Change for high metric of data


C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\src\computation\correlation.py:26: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return df.pct_change()


INFO:    Adjusting Correlation based on optimization strategy for high metric of data
SUCCESS:    Correlation Pipeline for high metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for high metric of data finished successfully

INFO:    Starting Correlation Pipeline for low metric of data
INFO:    Calculating PCT Change for low metric of data
INFO:    Calculating Correlation for PCT Change for low metric of data


...

INFO:    Adjusting Correlation based on optimization strategy for low metric of data
SUCCESS:    Correlation Pipeline for low metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for low metric of data finished successfully

INFO:    Starting Correlation Pipeline for close metric of data
INFO:    Calculating PCT Change for close metric of data
INFO:    Calculating Correlation for PCT Change for close metric of data


C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\src\computation\correlation.py:26: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return df.pct_change()


INFO:    Adjusting Correlation based on optimization strategy for close metric of data
SUCCESS:    Correlation Pipeline for close metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for close metric of data finished successfully

INFO:    Starting Correlation Pipeline for adj_close metric of data
INFO:    Calculating PCT Change for adj_close metric of data
INFO:    Calculating Correlation for PCT Change for adj_close metric of data


C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\src\computation\correlation.py:26: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return df.pct_change()


INFO:    Adjusting Correlation based on optimization strategy for adj_close metric of data
SUCCESS:    Correlation Pipeline for adj_close metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for adj_close metric of data finished successfully

SUCCESS: Correlation Pipeline complete!


## 2.3. Generate a portfolio Universe using the `create_portfolio_universe` function

In [19]:
anchors, candidate_portfolios = create_portfolio_universe(config_path=conf_path, save_data=True, verbose=True)

INFO: Loading Portfolio Params
SUCCESS: Loaded and Validated Portfolio Params

SUCCESS: Loaded the latest correlation Matrix

INFO: Calculating the Global Correlation for all securities in correlation matrix
INFO: Filtering global correlation to loowest globally correlated 5 anchor points
SUCCESS: Global Correlation for all securities caluclated

INFO: Finding Optimal Buckets
INFO: Loading Config Params for Portfolio Selection

SUCCESS: Loaded Config Params for Portfolio Selection

INFO: Creating portfolio for anchor: LIQUIDETF.NS

SUCCESS: Portfolio Created for anchor: LIQUIDETF.NS
INFO: Termination achieved, exhaustion of low correlation pairs below corr: 0.2
INFO: Portfolio Size: 8

INFO: Creating portfolio for anchor: IBMFNIFTY.NS

SUCCESS: Portfolio Created for anchor: IBMFNIFTY.NS
INFO: Termination achieved, exhaustion of low correlation pairs below corr: 0.2
INFO: Portfolio Size: 8

INFO: Creating portfolio for anchor: EBBETF0425.NS

SUCCESS: Portfolio Created for anchor: EBBETF